# 7. Aplicação

## Funções utilizadas

In [2]:
# Funções utilizadas
import os
import cv2
from glob import glob
from PIL import Image
import time


# Salva vídeo
def save_uploaded_video(uploadedfile):
    with open(os.path.join("data/external/","video_upload.mp4"),"wb") as f:
            f.write(uploadedfile.getbuffer())

# Caminho do vídeo
def caminho_video():
    return glob('data/external/*.mp4')[0]

# Visualiza vídeo
def play_video():
    return open(glob('data/external/*.mp4')[0], 'rb')

# Caminho das faces
def caminho_faces():
    return glob('data/interim/faces/*.jpg')

# Exclusão de arquivos importados
def limpar():
    filelist = [ f for f in os.listdir('data/external/') if (f.endswith(".mp4") or f.endswith(".mpg4"))]
    for f in filelist:
        os.remove(os.path.join('data/external/', f))
    filelist = [ f for f in os.listdir('data/interim/faces/') if (f.endswith(".jpg"))]
    for f in filelist:
        os.remove(os.path.join('data/interim/faces/', f))
    filelist = [ f for f in os.listdir('data/interim/') if (f.endswith(".gif"))]
    for f in filelist:
        os.remove(os.path.join('data/interim/', f))

# Captura de face:
face_cascade = cv2.CascadeClassifier('src/features/haarcascades/haarcascade_frontalface_default.xml')

def ROI(img):
    offset = 30 
    face_img = img.copy()
    face_rects = face_cascade.detectMultiScale(face_img,scaleFactor=1.3, minNeighbors=5) 
    for (x,y,w,h) in face_rects: 
        roi = face_img[y-offset:y+h+offset,x-offset:x+w+offset] 
    return roi

def save_faces(arquivo_video):
    cap = cv2.VideoCapture(arquivo_video)
    ret,frame = cap.read()
    count = 0
    try:
        while count < 10:
            cap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))   
            ret,frame = cap.read()
            image0 = frame
            image0 = ROI(image0)
            arquivo = arquivo_video.replace("data/external","").replace(".mp4","")
            arquivo_nome = "data/interim/faces/"+ arquivo + "_" + str(count) + ".jpg"
            cv2.imwrite(arquivo_nome,image0)
            count = count + 1
    except:
        pass


def gera_gif():
    frames = [Image.open(image) for image in glob('data/interim/faces/*jpg')]
    frame_one = frames[0]
    frame_one.save("data/interim/analise.gif", format="GIF", append_images=frames,
               save_all=True, duration=300, loop=0)

## Streamlit

In [ ]:
# Aplicação Streamlit
import os
import sys
import streamlit as st
from PIL import Image
from glob import glob
path_funcoes = os.path.abspath("src/app")
sys.path.append(path_funcoes)
import funcoes
path_modelo = os.path.abspath("src/models")
sys.path.append(path_modelo)
import modelo_final

# configuracao da página
st.set_page_config(page_title='Detector de DeepFakes', page_icon='🔍️', layout="wide", 
                   initial_sidebar_state="auto", menu_items=None)
""" # Detector de DeepFake"""
tab1, tab2= st.tabs(["Aplicação", "Sobre"])
with tab1:
  if st.button("Nova análise"): 
    funcoes.limpar()
    uploaded_video = None
  ## Caixa de Upload
  with st.form("caixa_upload", clear_on_submit=True):
    uploaded_video = st.file_uploader("Faça o upload de um vídeo em .mp4", accept_multiple_files=False)
    upcol1,upcol2,upcol3= st.columns([2,2,1])
    with upcol1:
      botao_enviar = st.form_submit_button("Enviar vídeo e analisar.")
    with upcol3:
       """\n"""
       st.write("Exemplos para testes :  [  🎥](https://drive.google.com/drive/folders/1g2j6IwiFAexctpHRCXyRjkLHb-lXUHPL?usp=sharing)")
       
    if botao_enviar and uploaded_video is not None:
      if uploaded_video.name.split('.')[1] == 'mp4' or uploaded_video.name.split('.')[1] == 'mpg4':
        funcoes.save_uploaded_video(uploaded_video)

  total_arquivos_mp4 = len(glob("data/external/*.mp4"))
  if total_arquivos_mp4 > 0:
    try:
      if uploaded_video.name.split('.')[1] == 'mp4' or uploaded_video.name.split('.')[1] == 'mpg4':
        funcoes.save_uploaded_video(uploaded_video)
        video_file = funcoes.play_video()
        video_bytes = video_file.read()
        st.write("Upload concluído.")
        st.video(video_bytes)
        """ *** """
        """ ### Análise de Frames: """
        funcoes.save_faces(funcoes.caminho_video())
        analise = modelo_final.roda_modelo()
        frames = analise[0]
        real = analise[1]
        fake = analise[2]
        final_real = analise[3]
        final_fake = analise[4]
      if uploaded_video is not None:
        faces_list = funcoes.caminho_faces()
        idx = 0 
        for _ in range(len(faces_list)-1): 
            cols = st.columns(5) 
            
            if idx < len(faces_list): 
                cols[0].image(faces_list[idx], width=150, caption=("""_REAL_:"""+str(real[idx].round(4)),"""_FAKE_:"""+str(fake[idx].round(4))))
            idx+=1
            if idx < len(faces_list):
                cols[1].image(faces_list[idx], width=150, caption=("""_REAL_:"""+str(real[idx].round(4)),"""_FAKE_:"""+str(fake[idx].round(4))))
            idx+=1
            if idx < len(faces_list):
                cols[2].image(faces_list[idx], width=150, caption=("""_REAL_:"""+str(real[idx].round(4)),"""_FAKE_:"""+str(fake[idx].round(4))))
            idx+=1 
            if idx < len(faces_list): 
                cols[3].image(faces_list[idx], width=150, caption=("""_REAL_:"""+str(real[idx].round(4)),"""_FAKE_:"""+str(fake[idx].round(4))))
                idx = idx + 1
            if idx < len(faces_list): 
              cols[4].image(faces_list[idx], width=150, caption=("""_REAL_:"""+str(real[idx].round(4)),"""_FAKE_: """+str(fake[idx].round(4))))
              idx = idx + 1
            else:
                break
        """ *** """
        """ ### Resultado Final: """
        funcoes.gera_gif()
        col1,col2 = st.columns(2)
        with col1:
          st.image('data/interim/analise.gif')
        with col2:
          """\n"""   
          """\n"""   
          """\n"""   
          """\n"""   
          #final_real = 5
          #final_fake = 10
          subcol1,subcol2= st.columns([1,4])
          if final_real > final_fake:
              with subcol1:
                st.image('references/images/resultado_real.png')
              with subcol2:
                """\n""" 
                """ #### REAL"""
          elif final_real == final_fake:
              """ #### INCONCLUSIVO """
          else:
              with subcol1:
                st.image('references/images/resultado_fake.png')
              with subcol2:
                """\n""" 
                """ #### DEEP FAKE """
          """\n"""   
          st.write("Probabilidade de ser Real: ",round((final_real*100),4))
          st.write("Probabilidade de ser um Deep Fake: ",round((final_fake*100),4))
      else:
        st.write('Arquivo não compatível. Por favor refaça o upload de um arquivo de vídeo em .mp4.')
    except:
       st.write('Arquivo não compatível. Por favor refaça o upload de um arquivo de vídeo em .mp4.')
       
with tab2:
  """Projeto de conclusão do curso MBA em Machine Learning da [:green[XPE]](https://www.xpeducacao.com.br/).
  \n - Autor: [:green[Diego Abreu]](https://www.linkedin.com/in/diego-s-abreu/)
  \n- Repositório: [:green[Github]](https://github.com/DiegoAbreu/Proj-Aplicado-XPE-MBA-Machine-Learning)"""